In [ ]:
%load_ext autoreload
%autoreload 2
import sike
import numpy as np

In [ ]:
# sike.setup(elements = ["H"], savedir = "..")

In [ ]:
num_x = 10
Te = np.linspace(1,10,num_x)
ne = np.ones(num_x)
c = sike.SIKERun(Te=Te, ne=ne, element="Ne", resolve_j = False, resolve_l = False, evolve=False)

In [ ]:
c.run()